# MetaMind CoT - Claude

In [52]:
import json
from pathlib import Path
from typing import Dict

PROMPT_PATH = Path("MetaMindCoT.json") # U
def build_system_prompt(prompt_cfg: Dict[str, str]) -> str:
    parts = [
        prompt_cfg.get("sys_instruction", "").strip(),
        prompt_cfg.get("thinking_steps", "").strip(),
        prompt_cfg.get("return_format", "").strip(),
    ]
    return "\n\n".join([p for p in parts if p])

with PROMPT_PATH.open("r", encoding="utf-8") as f:
    prompt_cfg = json.load(f)

# build system prompt string
system_prompt = build_system_prompt(prompt_cfg)
# sanity check
# print(system_prompt)

EVAL_JSONL_PATH = Path("output.jsonl")
def load_jsonl(path: Path):
    with path.open("r", encoding="utf-8") as f:
        return [json.loads(line) for line in f if line.strip()]
benchmarks = load_jsonl(EVAL_JSONL_PATH)
print(f"Loaded {len(benchmarks)} benchmark entries.")
print("Sample entry:", benchmarks[0])

def make_messages(benchmark: Dict, system_prompt: str):
    '''
    Changed to fit Anthropic messages conventions
    returns: "messages" var to go into llm_query
    '''
    context = benchmark.get("STORY", "").strip()
    question = benchmark.get("QUESTION", "").strip()

    options = "\n".join(
        f"{key}: {benchmark.get(key, '')}"
        for key in benchmark
        if key.startswith("OPTION-")
    )

    system_message = {
        "role": "system",
        "content": system_prompt
    }

    user_message = {
        "role": "user",
        "content": (
            f"id: {benchmark.get('INDEX')}\n\n"
            f"Story:\n{context}\n\n"
            f"Question:\n{question}\n\n"
            f"Options:\n{options}"
        )
    }

    return [system_message, user_message]

print("Sample messages:", make_messages(benchmarks[0], system_prompt))

# Anthropic Claude in place of other models
from anthropic import Anthropic
import os
from dotenv import load_dotenv

load_dotenv()
client = Anthropic(
    api_key=os.environ.get("ANTHROPIC_API_KEY"), 
)   

def llm_query(messages):
    '''
    takes in messages: a list with [system_message, user_message]

    system_message = {
        "role": "system",
        "content": system_prompt
    }

    user_message = {
        "role": "user",
        "content": ()
    }
    ''' 

    system_prompt = messages[0]['content'] # extract system prompt
    user_messages = [messages[1]]

    response = client.messages.create(
        max_tokens=2048,
        system=system_prompt,
        messages=user_messages,
        model="claude-sonnet-4-5-20250929",
        output_config={
            "format": {
                "type": "json_schema",
                "schema": {
                    "type": "object",
                    "properties": {
                        "pred": {
                            "type": "string",
                            "enum": ["A", "B", "C", "D"],
                            "description": "The predicted answer"
                        }
                    },
                    "required": ["pred"],
                    "additionalProperties": False
                }
            }
        }
    )
    return response.content[0].text

Loaded 600 benchmark entries.
Sample entry: {'ABILITY': 'Belief: Location false beliefs', 'INDEX': 1, 'STORY': 'Xiaogang and Xiaoming are wandering in the bedroom, they see a handbag, a briefcase, and a backpack, they find cabbage in the handbag, Xiaoming leaves the bedroom, Xiaogang moves the cabbage to the backpack.', 'QUESTION': 'Where is the cabbage now?', 'OPTION-A': 'Backpack', 'OPTION-B': 'Handbag', 'OPTION-C': 'Tote bag', 'OPTION-D': 'Briefcase', 'ANSWER': 'A'}
Sample messages: [{'role': 'system', 'content': 'You are a structured social reasoning system that models human social cognition using explicit Theory-of-Mind inference and norm-aware reasoning. Given a user input and optional context, infer latent mental states, refine them using social and ethical considerations, and generate a grounded response.\n\nOutput ONLY valid JSON.\n\nInstructions:\n1) Generate 3-5 distinct hypotheses about the user\'s latent mental states. Each hypothesis must include: id, type ∈ ["belief","de

In [53]:
# Extract answers from raw_response and store to pandas dataframe
import pandas as pd
from tqdm import tqdm
rows = []

for i, benchmark in tqdm(enumerate(benchmarks[:100])):
    messages = make_messages(benchmark, system_prompt)

    raw_response = llm_query(messages)      # string
    try:
        response = json.loads(raw_response) # dict
    except Exception:
        response = {} 

    print("=" * 50)
    print("Raw response:")
    print(raw_response)
    print("=" * 50)

    correct_answer = benchmark.get("ANSWER", "").strip()
    pred = response.get("pred", "").strip()

    result = (pred == correct_answer)

    rows.append({
        "id": i,
        "benchmark_id": benchmark.get("INDEX"),
        "pred": pred,
        "gold": correct_answer,
        "correct": result,
        "raw_response": raw_response
    })

MetaMindCoT = pd.DataFrame(rows)
MetaMindCoT.to_csv("eval_result/MetaMindCoT_Claude.csv", index=False)

1it [00:01,  1.29s/it]

Raw response:
{"pred":"A"}


2it [00:02,  1.39s/it]

Raw response:
{"pred":"B"}


3it [00:04,  1.49s/it]

...

Raw response:
{"pred":"C"}


99it [06:32,  2.58s/it]

Raw response:
{"pred":"C"}


100it [06:33,  3.94s/it]

Raw response:
{"pred":"D"}


# General Prompt (No MetaMind-style prompt)

In [54]:
rows = []
for i, benchmark in tqdm(enumerate(benchmarks[:100])):
    general_prompt = (
    "You are a socially intelligent assistant answering multiple-choice reasoning questions.\n"
    "Return ONLY valid JSON.\n\n"
    "JSON schema:\n"
    "{\n"
    "  \"pred\": \"A|B|C|D\"\n"
    "}\n\n"
    "Rules:\n"
    "- pred must be exactly one of A, B, C, D.\n"
    "- Do not output anything outside the JSON."
)
    messages = make_messages(benchmark, general_prompt)

    raw_response = llm_query(messages)      # string

    print("=" * 50)
    print("Raw response:")
    print(raw_response)
    print("=" * 50)

    try:
        response = json.loads(raw_response) # dict
    
    except Exception as e:
        print(f"JSON parse error: {e}")
        response = {}
        
    correct_answer = benchmark.get("ANSWER", "").strip()
    pred = response.get("pred", "").strip()

    result = (pred == correct_answer)

    rows.append({
        "id": i,
        "benchmark_id": benchmark.get("INDEX"),
        "pred": pred,
        "gold": correct_answer,
        "correct": result,
        "raw_response": raw_response
    })

GenClaude = pd.DataFrame(rows)
GenClaude.to_csv("eval_result/Gen_Claude.csv", index=False)

1it [00:01,  1.17s/it]

Raw response:
{"pred": "A"}


2it [00:02,  1.09s/it]

Raw response:
{"pred": "B"}


3it [00:03,  1.25s/it]

...

Raw response:
{"pred":"C"}


99it [02:11,  1.42s/it]

Raw response:
{"pred":"C"}


100it [02:13,  1.33s/it]

Raw response:
{"pred":"D"}


In [55]:
import matplotlib.pyplot as plt
# Print percentage summary of correct predictions for both Prompt attempts
acc_summary = {
    "GenClaude": GenClaude["correct"].mean().astype(float),
    "MetaMind CoT": MetaMindCoT["correct"].mean().astype(float)
}
print(acc_summary)


{'GenClaude': np.float64(0.98), 'MetaMind CoT': np.float64(0.96)}


In [56]:
MetaMindCoT = pd.DataFrame(rows)
MetaMindCoT


,id,benchmark_id,pred,gold,correct,raw_response
0,0,1,A,A,True,"{""pred"": ""A""}"
1,1,2,B,B,True,"{""pred"": ""B""}"
2,2,3,A,A,True,"{""pred"":""A""}"
3,3,4,D,D,True,"{""pred"":""D""}"
4,4,5,B,B,True,"{""pred"": ""B""}"
...,...,...,...,...,...,...
95,95,6,A,A,True,"{""pred"": ""A""}"
96,96,1,C,C,True,"{""pred"": ""C""}"
97,97,2,C,C,True,"{""pred"":""C""}"
98,98,3,C,C,True,"{""pred"":""C""}"
